In [61]:
from tabulate import tabulate
from operator import itemgetter
from pprint import pprint

In [62]:
import population

#print(population.search('new york', 'nassau'))

from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

In [63]:
county_data = open("../covid-19-data/us-counties.csv")

In [64]:
def search_v2(data, s, c=False):
    if c == '':
        c = False
    p_data = population.search(s, c)

    if p_data == None:
        print("No match for that geography.")
        return
    else:
        #pass
        pprint(p_data)
    header = ['Date', 'Cases', 'Deaths']
    c_data = []
    prior_c = 0
    prior_d = 0
    row_count = 0
    for line in data:
        row_count += 1
        include = False
        row = line.strip().split(',')
        if len(row) == 6:
            if c is not False:
                if c.lower() == "new york city":
                    if row[1] == 'New York City':
                        include = True
            if row[3] == p_data['composite_fips']:
                include = True
            if include:
                delta = int(row[4]) - prior_c
                delta_d = int(row[5]) - prior_d
                if prior_c > 0:
                    delta_p = float("{:.2f}".format((delta / prior_c) * 100))
                else:
                    delta_p = '-'
                    
                if prior_d > 0:
                    delta_dp = float("{:.4f}".format((delta_d / prior_d) * 100))
                else:
                    delta_dp = '-'
                    
                if delta > -1:
                    c_data.append([row[0], row[4], delta, delta_p, row[5], delta_d, delta_dp])
                prior_c = int(row[4])
                prior_d = int(row[5])
    print("Row Count: ", row_count)
    if c_data == []:
        print("No case data for that geography.")
        return
    perc_infected = float("{:.4f}".format(int(c_data[-1][1]) / int(p_data['POPESTIMATE2019']) * 100))
    mortality_rate = float("{:.4f}".format(int(c_data[-1][4]) / int(p_data['POPESTIMATE2019']) * 100))
    case_mortality = float("{:.2f}".format(int(c_data[-1][4]) / int(c_data[-1][1]) * 100))
    print("POPESTIMATE2019", p_data['POPESTIMATE2019'])
    print("Case Rate: ", perc_infected, '%')
    print("Mortality (vs. total population): ", mortality_rate, '%')
    print("Case Mortality %: ", case_mortality, '%')
    print("2019 Average deaths/day: ", float("{:.1f}".format(int(p_data['DEATHS2019']) / 365)))
    if len(c_data) >= 7:
        print( "Last 7 day case increase  (%):", float("{:.2f}".format(((int(c_data[-1][1]) - int(c_data[-8][1]))/ int(c_data[-8][1])) * 100 )),  (int(c_data[-1][1]) - int(c_data[-8][1])))    
        if int(c_data[-8][4]) > 0:
            print( "Last 7 day death increase (%):", float("{:.2f}".format(((int(c_data[-1][4]) - int(c_data[-8][4]))/ int(c_data[-8][4])) * 100 )),(int(c_data[-1][4]) - int(c_data[-8][4])) )
    print(tabulate(c_data[-30:], headers=['Date', 'Cases', 'Delta', '% Inc', 'Deaths', 'Delta', '% Inc']))

In [65]:

s = input("State: ").strip()
c = input("County: ").strip()
search_v2(county_data, s, c)

State: Iowa
County: Dallas
{'COUNTY': '049',
 'CTYNAME': 'Dallas County',
 'DEATHS2019': '507',
 'POPESTIMATE2019': '93453',
 'RDEATH2019': '5.5287177083',
 'REGION': '2',
 'STATE': '19',
 'STNAME': 'Iowa',
 'SUMLEV': '050',
 'composite_fips': '19049'}
Row Count:  331180
POPESTIMATE2019 93453
Case Rate:  1.5141 %
Mortality (vs. total population):  0.0332 %
Case Mortality %:  2.19 %
2019 Average deaths/day:  1.4
Last 7 day case increase  (%): 10.2 131
Last 7 day death increase (%): 6.9 2
Date          Cases    Delta    % Inc    Deaths    Delta    % Inc
----------  -------  -------  -------  --------  -------  -------
2020-06-13     1019       14     1.39        26        0   0
2020-06-14     1026        7     0.69        26        0   0
2020-06-15     1029        3     0.29        26        0   0
2020-06-16     1031        2     0.19        28        2   7.6923
2020-06-17     1046       15     1.45        29        1   3.5714
2020-06-18     1051        5     0.48        28       -1  -3.